In [ ]:
import pandas as pd
import numpy as np
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from flask import Flask, render_template, request, jsonify, send_from_directory, redirect, url_for, flash, session
from flask_socketio import SocketIO, emit
from flask_sqlalchemy import SQLAlchemy
from datetime import datetime
from functools import wraps

# 데이터 로드 및 전처리
BASE_PROJECT_PATH = os.path.abspath(os.path.dirname(__file__))
lck = pd.read_csv(os.path.join(BASE_PROJECT_PATH, 'lck_num.csv'))

lck['GRB%'].fillna(0, inplace=True)
lck['ELD%'].fillna(0, inplace=True)
lck['BN%'].fillna(0, inplace=True)

# 'K', 'D', 'PPG' 컬럼 숫자 변환 및 NaN 처리
lck['K'] = pd.to_numeric(lck['K'], errors='coerce')
lck['K'] = lck['K'].fillna(lck['K'].mean())
lck['D'] = pd.to_numeric(lck['D'], errors='coerce')
lck['D'] = lck['D'].fillna(lck['D'].mean())
lck['PPG'] = pd.to_numeric(lck['PPG'], errors='coerce')
lck['PPG'] = lck['PPG'].fillna(0)

# Label Encoding 적용
lb = LabelEncoder()
for col in ['Team', 'GSPD', 'FB%', 'FT%', 'F3T%', 'HLD%', 'GRB%', 'FD%', 'DRG%', 'ELD%', 'FBN%', 'BN%', 'LNE%', 'JNG%']:
    lck[col] = lb.fit_transform(lck[col])

# 추가 NaN 값 처리
lck['KD'] = lck['KD'].fillna(lck['KD'].mean())
lck['CKPM'] = lck['CKPM'].fillna(lck['CKPM'].mean())
lck['GPR'] = lck['GPR'].fillna(lck['GPR'].mean())
lck['EGR'] = lck['EGR'].fillna(lck['EGR'].mean())
lck['MLR'] = lck['MLR'].fillna(lck['MLR'].mean())
lck['GD15'] = lck['GD15'].fillna(lck['GD15'].mean())
lck['WPM'] = lck['WPM'].fillna(lck['WPM'].mean())
lck['CWPM'] = lck['CWPM'].fillna(lck['CWPM'].mean())
lck['WCPM'] = lck['WCPM'].fillna(lck['WCPM'].mean())

# 종속 변수 생성 및 데이터 분리
lck['GP_W'] = lck['W'] / lck['GP']
y_data = lck['GP_W']
x_data = lck.drop(['GP_W'], axis=1)

# 훈련 및 테스트 데이터 분리
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, random_state=42, test_size=0.2)

# GradientBoostingRegressor 모델 학습
gr = GradientBoostingRegressor(random_state=42, max_depth=20)
gr.fit(x_train, y_train)

# 팀 이름과 인코딩된 값 매핑 (하드코딩된 값 사용)
teams = {
    "BNK FEARX": 0, "BRION": 1, "DN Freecs": 2, "DRX": 3, "Dplus KIA": 4,
    "Gen.G": 5, "Hanwha Life Esports": 6, "KT Rolster": 7, "Liiv SANDBOX": 8,
    "Nongshim RedForce": 9, "OKSavingsBank BRION": 10, "T1": 11
}

from flask import Flask, render_template, request, jsonify, send_from_directory, redirect, url_for, flash, session
from flask_socketio import SocketIO, emit
from flask_sqlalchemy import SQLAlchemy
import numpy as np
import json
from datetime import datetime
import os
from functools import wraps

# --- 중요한 변경: 프로젝트 기본 경로를 명시적으로 설정 ---
# 이 경로를 여러분의 'project_lck_250605' 폴더의 실제 절대 경로로 직접 지정해주세요.
# 예: "C:/Users/PHANTOM/Desktop/a9a9a9/project_lck_250605"


# Flask 애플리케이션 초기화
# template_folder와 static_folder 경로를 BASE_PROJECT_PATH를 기반으로 설정
app = Flask(__name__,
            template_folder=os.path.join(BASE_PROJECT_PATH, "templates"),
            static_folder=os.path.join(BASE_PROJECT_PATH, "static"))


# SocketIO 초기화에 필요한 SECRET_KEY 설정
app.config['SECRET_KEY'] = 'your_secret_key_for_socketio'

# --- 데이터베이스 설정 추가 ---
# site.db 파일 경로를 BASE_PROJECT_PATH를 기반으로 설정
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///' + os.path.join(BASE_PROJECT_PATH, 'site.db')
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app) # SQLAlchemy 객체 초기화


# --- 데이터베이스 모델 정의 (게시글) ---
class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100), nullable=False)
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(20), nullable=False, default='익명')
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    def __repr__(self):
        return f"Post('{self.title}', '{self.date_posted}')"

# 댓글 구현
class Comment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.id'), nullable=False)  # 게시글 ID와 연결
    content = db.Column(db.Text, nullable=False)
    author = db.Column(db.String(20), nullable=False, default="익명")
    date_posted = db.Column(db.DateTime, nullable=False, default=datetime.utcnow)

    post = db.relationship('Post', backref=db.backref('comments', lazy=True))  # 댓글과 게시글 연결

    def __repr__(self):
        return f"Comment('{self.content}', '{self.date_posted}')"

@app.route('/predict', methods=['POST'])
def predict():
    data = request.json  # JavaScript에서 받은 JSON 데이터
    print("데이터확인>> ",data)
    team_name = data.get('team')  # 받은 팀 이름
    
    if team_name not in teams:
        return jsonify({"error": "잘못된 팀 이름입니다."})

    team_encoded = teams[team_name]  # 팀 이름을 숫자로 변환
    prediction = gr.predict(x_train[x_train['Team'] == teams[team_name]]).mean()*100  # 예측 수행

    print("팀 숫자 변환 값 :", team_encoded)

    return jsonify({
        "team_name": team_name,
        "win_rate": round(prediction, 2)
    })

# --- SocketIO 초기화 ---
socketio = SocketIO(app)



# --- 라우트 정의 ---

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/Simulator')
def Simulator():
    return render_template('Simulator.html')

@app.route('/bnk_fearx')
def bnk_fearx():
    return render_template('BNK FEARX.html')

@app.route('/dn_freecs')
def dn_freecs():
    return render_template('DN Freecs.html')

@app.route('/dplus_kia')
def dplus_kia():
    return render_template('Dplus KIA.html')

@app.route('/drx')
def drx():
    return render_template('DRX.html')

@app.route('/gen_g')
def gen_g():
    return render_template('Gen.G.html')

@app.route('/hanwha_life_esports')
def hanwha_life_esports():
    return render_template('Hanwha Life Esports.html')

@app.route('/kt_rolster')
def kt_rolster():
    return render_template('KT Rolster.html')

@app.route('/nongshim_redforce')
def nongshim_redforce():
    return render_template('Nongshim RedForce.html')

@app.route('/oksavingsbank_brion')
def okSavingsBank_brion():
    return render_template('BRION.html')

@app.route('/player_comparison')
def player_comparison():
    return render_template('Player comparison.html')

@app.route('/T1')
def T1():
    return render_template('T1.html')

@app.route('/static/<path:filename>')
def serve_static(filename):
    return send_from_directory(app.static_folder, filename)


# --- SocketIO 이벤트 핸들러 ---
@app.route('/chat')
def chat():
    return render_template('chat.html')

@socketio.on('connect')
def test_connect():
    print('Client connected')
    emit('status', {'msg': '새로운 사용자가 입장했습니다.'}, broadcast=True)

@socketio.on('disconnect')
def test_disconnect():
    print('Client disconnected')
    emit('status', {'msg': '사용자가 퇴장했습니다.'}, broadcast=True)

@socketio.on('message')
def handle_message(data):
    print('received message: ' + str(data))
    emit('message', data, broadcast=True)

# --- 게시판 관련 라우트 ---

@app.route('/board')
def board():
    posts = Post.query.order_by(Post.date_posted.desc()).all()
    return render_template('board.html', posts=posts)

@app.route('/board/new', methods=['GET', 'POST'])
def new_post():
    if request.method == 'POST':
        title = request.form['title']
        content = request.form['content']
        author = request.form.get('author', '익명')

        post = Post(title=title, content=content, author=author)
        db.session.add(post)
        db.session.commit()
        return redirect(url_for('board'))
    return render_template('create_post.html')

@app.route('/board/<int:post_id>')
def post_detail(post_id):
    post = Post.query.get_or_404(post_id)
    return render_template('post_detail.html', post=post)

@app.route('/board/<int:post_id>/update', methods=['GET', 'POST'])
def update_post(post_id):
    post = Post.query.get_or_404(post_id)
    if request.method == 'POST':
        post.title = request.form['title']
        post.content = request.form['content']
        post.author = request.form.get('author', '익명')
        db.session.commit()
        return redirect(url_for('post_detail', post_id=post.id))
    return render_template('update_post.html', post=post)

@app.route('/board/<int:post_id>/delete', methods=['POST'])
def delete_post(post_id):
    post = Post.query.get_or_404(post_id)
    Comment.query.filter_by(post_id=post_id).delete()
    db.session.delete(post)
    db.session.commit()
    flash('게시글이 삭제되었습니다.', 'success')
    return redirect(url_for('board'))


@app.route('/board/<int:post_id>/comment', methods=['POST'])
def add_comment(post_id):
    post = Post.query.get_or_404(post_id)
    content = request.form['content']
    author = request.form.get('author', '익명')

    comment = Comment(post_id=post.id, content=content, author=author)
    db.session.add(comment)
    db.session.commit()

    return redirect(url_for('post_detail', post_id=post.id))


    
# --- 관리자 계정 정보 (하드코딩) ---
ADMIN_USERNAME = "admin"
ADMIN_PASSWORD = "1234"

# --- 관리자 로그인 ---
def admin_required(f):
    @wraps(f)
    def decorated_function(*args, **kwargs):
        if 'admin_logged_in' not in session or not session['admin_logged_in']:
            flash('관리자 권한이 필요합니다.', 'error')
            return redirect(url_for('admin_login'))
        return f(*args, **kwargs)
    return decorated_function
    
# --- 관리자 로그인 및 로그아웃 ---
@app.route('/admin/login', methods=['GET', 'POST'])
def admin_login():
    if request.method == 'POST':
        username = request.form['username']
        password = request.form['password']
        if username == ADMIN_USERNAME and password == ADMIN_PASSWORD:
            session['admin_logged_in'] = True
            flash('관리자로 로그인되었습니다.', 'success')
            return redirect(url_for('admin_dashboard'))
        else:
            flash('잘못된 아이디 또는 비밀번호입니다.', 'error')
    return render_template('admin_login.html')

@app.route('/admin/logout')
@admin_required
def admin_logout():
    session.pop('admin_logged_in', None)
    flash('관리자 로그아웃되었습니다.', 'info')
    return redirect(url_for('admin_login'))

# --- 관리자 대시보드 ---
@app.route('/admin/dashboard')
@admin_required
def admin_dashboard():
    posts = Post.query.order_by(Post.date_posted.desc()).all()
    return render_template('admin_dashboard.html', posts=posts)

# --- 관리자 게시물 삭제 라우트 ---
@app.route('/admin/posts/<int:post_id>', methods=['POST'])
@admin_required
def admin_delete_post(post_id):
    post = Post.query.get_or_404(post_id)
    
    # 게시물에 달린 모든 댓글 먼저 삭제
    Comment.query.filter_by(post_id=post_id).delete()
    db.session.delete(post)
    db.session.commit()
    flash('게시글 및 관련 댓글이 삭제되었습니다.', 'success')
    return redirect(url_for('admin_dashboard'))

# --- 게시물 상세 정보 및 댓글 불러오기 ---
@app.route('/api/admin/posts/<int:post_id>')
@admin_required
def api_admin_post_detail(post_id):
    post = Post.query.get_or_404(post_id)
    comments = Comment.query.filter_by(post_id=post_id).order_by(Comment.date_posted.asc()).all()

    post_data = {
        'id': post.id,
        'title': post.title,
        'content': post.content,
        'author': post.author,
        'date_posted': post.date_posted.strftime('%Y-%m-%d %H:%M'),
    }
    comments_data = []
    for comment in comments:
        comments_data.append({
            'id': comment.id,
            'content': comment.content,
            'author': comment.author,
            'date_posted': comment.date_posted.strftime('%Y-%m-%d %H:%M')
        })
    post_data['comments'] = comments_data
    return jsonify(post_data)

# --- 관리자 댓글 삭제 ---
@app.route('/api/admin/comments/<int:comment_id>/delete', methods=['POST'])
@admin_required
def api_admin_delete_comment(comment_id):
    comment = Comment.query.get(comment_id)
    if comment:
        db.session.delete(comment)
        db.session.commit()
        return jsonify({'success': '댓글이 삭제되었습니다.'})
    return jsonify({'error': '댓글을 찾을 수 없습니다.'}), 404


# --- 애플리케이션 실행 ---
if __name__ == '__main__':
    # Flask 앱 컨텍스트 내에서 데이터베이스 테이블을 생성합니다.
    # 이 부분은 'app.py' 파일이 직접 실행될 때만 동작합니다.
    with app.app_context():
        db.create_all() # 데이터베이스 테이블 생성 (최초 1회만 실행)

    # Socket.IO를 포함하여 Flask 서버를 실행합니다.
    socketio.run(app, debug=True, use_reloader=False, host='0.0.0.0', port=5000, allow_unsafe_werkzeug=True)

print(app.run())

C:\Users\User\AppData\Local\Temp\ipykernel_26772\3966039080.py:16: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  lck['GRB%'].fillna(0, inplace=True)
C:\Users\User\AppData\Local\Temp\ipykernel_26772\3966039080.py:17: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when do

 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://192.168.0.74:5000
Press CTRL+C to quit
192.168.0.74 - - [11/Jun/2025 16:19:44] "GET /board HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /admin/dashboard HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/T1.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/kt%20rolster.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/OKSavingsBank%20BRION.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/images/Nongshim%20RedForce.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /static/css/style.css HTTP/1.1" 404 -
192.168.0.74 - - [11/Jun/2025 16:19:45] "GET /socket.io/?EIO=4&transport=po

Client connected


GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:41] "GET / HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /T1 HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/DRX.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/kt%20rolster.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/OKSavingsBank%20BRION.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:23:43] "GET /static/images/Nongshim%20RedForce.pn

팀 숫자 변환 값 : 11
Client connected


192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /socket.io/?EIO=4&transport=websocket&sid=vEjqLvMXvgFXQxc2AAAA HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /board HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/BNK%20FEARX.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/DN%20freecs.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/Dplus%20KIA.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/Hanwha%20Life%20Esports.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/Gen.G.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/DRX.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /socket.io/?EIO=4&transport=polling&t=PTTu0P2 HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/T1.png HTTP/1.1" 304 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/images/kt%20rol

Client disconnected
Client connected


192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /admin/dashboard HTTP/1.1" 200 -
192.168.0.74 - - [11/Jun/2025 16:24:12] "GET /static/css/style.css HTTP/1.1" 404 -
192.168.0.74 - - [11/Jun/2025 16:24:26] "GET /socket.io/?EIO=4&transport=websocket&sid=0KIUUqVnUbDTsVtQAAAC HTTP/1.1" 200 -


Client disconnected
